이번에는 입출력의 단위를 단어 레벨(word-level)이 아닌 글자 레벨(character-level)로 변경해 RNN을 구현

## 1. 글자 단위 RNN 언어 모델(Char RNNLM)
* 이전 시점의 예측 글자를 다음 시점의 입력으로 사용하는 글자 단위 RNN 언어 모델을 구현
* 글자 단위를 입, 출력으로 사용하므로 임베딩층(embedding layer)을 사용하지 않고 구현

### 1) 데이터에 대한 이해와 전처리

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import urllib.request
from tensorflow.keras.utils import to_categorical

In [2]:
urllib.request.urlretrieve("http://www.gutenberg.org/files/11/11-0.txt", filename="./data/11-0.txt")
f = open('./data/11-0.txt', 'rb')
lines = []
for line in f:
    line=line.strip() # strip()을 통해 \r, \n을 제거한다.
    line=line.lower() # 소문자화
    line=line.decode('ascii','ignore') # \xe2\x80\x99 등과 같은 바이트 열 제거
    if len(line) > 0:
        lines.append(line)
        
f.close()

In [3]:
lines[:5]

['the project gutenberg ebook of alices adventures in wonderland, by lewis carroll',
 'this ebook is for the use of anyone anywhere in the united states and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. you may copy it, give it away or re-use it under the terms',
 'of the project gutenberg license included with this ebook or online at']

In [4]:
# 하나의 문자열로 통합
text = ' '.join(lines)
print('문자열의 길이 또는 총 글자의 개수: %d' % len(text))

문자열의 길이 또는 총 글자의 개수: 159484


In [5]:
print(text[:200])

the project gutenberg ebook of alices adventures in wonderland, by lewis carroll this ebook is for the use of anyone anywhere in the united states and most other parts of the world at no cost and with


In [6]:
# 글자 집합
char_vocab = sorted(list(set(text)))
vocab_size = len(char_vocab)
print('글자 집합의 크기 : {}'.format(vocab_size))

글자 집합의 크기 : 56


* 영어가 훈련 데이터일 때 대부분의 경우에서 글자 집합의 크기가 단어 집합을 사용했을 경우보다 집합의 크기가 현저히 작다는 특징이 있다.

In [7]:
# 글자 집합에 인덱스를 부여하고 전부 출력
char_to_index = dict((c,i) for i,c in enumerate(char_vocab))
print(char_to_index)

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '?': 26, '[': 27, ']': 28, '_': 29, 'a': 30, 'b': 31, 'c': 32, 'd': 33, 'e': 34, 'f': 35, 'g': 36, 'h': 37, 'i': 38, 'j': 39, 'k': 40, 'l': 41, 'm': 42, 'n': 43, 'o': 44, 'p': 45, 'q': 46, 'r': 47, 's': 48, 't': 49, 'u': 50, 'v': 51, 'w': 52, 'x': 53, 'y': 54, 'z': 55}


In [8]:
# 인덱스로부터 글자 리턴
index_to_char={}
for key, value in char_to_index.items():
    index_to_char[value] = key

In [9]:
# text 문자열로부터 다수의 문장 샘플들로 분리

seq_length = 60 # 문장의 길이를 60으로 한다.
n_samples = int(np.floor((len(text)-1)/seq_length)) # 문자열을 60등분한다. 그러면 즉, 총 샘플의 개수
print ('문장 샘플의 수 : {}'.format(n_samples))

문장 샘플의 수 : 2658


In [10]:
train_X = []
train_y = []

for i in range(n_samples):
    X_sample = text[i*seq_length: (i+1) * seq_length]
    X_encoded = [char_to_index[c] for c in X_sample] # 하나의 문장 샘플에 대해 정수 인코딩
    train_X.append(X_encoded)
    
    y_sample = text[i*seq_length + 1: (i+1)*seq_length + 1]
    y_encoded = [char_to_index[c] for c in y_sample]
    train_y.append(y_encoded)

In [11]:
print(train_X[0])

[49, 37, 34, 0, 45, 47, 44, 39, 34, 32, 49, 0, 36, 50, 49, 34, 43, 31, 34, 47, 36, 0, 34, 31, 44, 44, 40, 0, 44, 35, 0, 30, 41, 38, 32, 34, 48, 0, 30, 33, 51, 34, 43, 49, 50, 47, 34, 48, 0, 38, 43, 0, 52, 44, 43, 33, 34, 47, 41, 30]


In [12]:
print(train_y[0]) # train_X[0]에서 오른쪽으로 한 칸 쉬프트 된 문장

[37, 34, 0, 45, 47, 44, 39, 34, 32, 49, 0, 36, 50, 49, 34, 43, 31, 34, 47, 36, 0, 34, 31, 44, 44, 40, 0, 44, 35, 0, 30, 41, 38, 32, 34, 48, 0, 30, 33, 51, 34, 43, 49, 50, 47, 34, 48, 0, 38, 43, 0, 52, 44, 43, 33, 34, 47, 41, 30, 43]


글자 단위 RNN에서는 입력 시퀀스에 대해서 워드 임베딩을 하지 않습니다. 

In [13]:
# 원-핫 인코딩
train_X = to_categorical(train_X)
train_y = to_categorical(train_y)

In [14]:
print('train_X의 크기(shape) : {}'.format(train_X.shape)) # 원-핫 인코딩
print('train_y의 크기(shape) : {}'.format(train_y.shape)) # 원-핫 인코딩
# 샘플의 수(No. of samples)가 2,658개, 입력 시퀀스의 길이(input_length)가 60, 각 벡터의 차원(input_dim)이 56

train_X의 크기(shape) : (2658, 60, 56)
train_y의 크기(shape) : (2658, 60, 56)


### 2) 모델 설계하기

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

In [16]:
model = Sequential()
model.add(LSTM(256, input_shape=(None, train_X.shape[2]), return_sequences=True))
model.add(LSTM(256, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_y, epochs=80, verbose=2)

Epoch 1/80
84/84 - 23s - loss: 3.0735 - accuracy: 0.1818
Epoch 2/80
84/84 - 21s - loss: 2.7202 - accuracy: 0.2485
Epoch 3/80
84/84 - 24s - loss: 2.3723 - accuracy: 0.3336
Epoch 4/80
84/84 - 24s - loss: 2.2359 - accuracy: 0.3639
Epoch 5/80
84/84 - 27s - loss: 2.1328 - accuracy: 0.3917
Epoch 6/80
84/84 - 27s - loss: 2.0506 - accuracy: 0.4121
Epoch 7/80
84/84 - 25s - loss: 1.9840 - accuracy: 0.4279
Epoch 8/80
84/84 - 26s - loss: 1.9220 - accuracy: 0.4445
Epoch 9/80
84/84 - 28s - loss: 1.8704 - accuracy: 0.4579
Epoch 10/80
84/84 - 29s - loss: 1.8200 - accuracy: 0.4720
Epoch 11/80
84/84 - 30s - loss: 1.7729 - accuracy: 0.4854
Epoch 12/80
84/84 - 28s - loss: 1.7283 - accuracy: 0.4977
Epoch 13/80
84/84 - 28s - loss: 1.6958 - accuracy: 0.5046
Epoch 14/80
84/84 - 28s - loss: 1.6541 - accuracy: 0.5156
Epoch 15/80
84/84 - 28s - loss: 1.6196 - accuracy: 0.5240
Epoch 16/80
84/84 - 28s - loss: 1.5879 - accuracy: 0.5327
Epoch 17/80
84/84 - 27s - loss: 1.5542 - accuracy: 0.5416
Epoch 18/80
84/84 - 29s

In [18]:
def sentence_generation(model, length):
    ix = [np.random.randint(vocab_size)]  # 글자에 대한 0~vocab_size 사이의 난수 1개 랜덤 인덱스 생성
    y_char = [index_to_char[ix[-1]]] # 랜덤 인덱스로부터 글자 생성
    print(ix[-1],'번 글자',y_char[-1],'로 예측을 시작!')
    X = np.zeros((1, length, vocab_size))  # (1, length, 55) 크기의 X 생성. 즉, LSTM의 입력 시퀀스 생성
    
    for i in range(length):
        X[0][i][ix[-1]] = 1  # X[0][i][예측한 글자의 인덱스] = 1, 즉, 예측 글자를 다음 입력 시퀀스에 추가
        print(index_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:,:i+1,:])[0], 1)
        y_char.append(index_to_char[ix[-1]])
    return ('').join(y_char)

In [19]:
sentence_generation(model, 100)

27 번 글자 [ 로 예측을 시작!
[late cook a mine. the next wainting up the capering wit, an accoudmed to know. wet? as she cauged t

'[late cook a mine. the next wainting up the capering wit, an accoudmed to know. wet? as she cauged th'

##  2. 글자 단위 RNN(Char RNN)으로 텍스트 생성하기
다 대 일(many-to-one) 구조의 RNN

### 1) 데이터에 대한 이해와 전처리

In [20]:
import numpy as np
from tensorflow.keras.utils import to_categorical

In [21]:
# 가상의 노래 가사
text='''
I get on with life as a programmer,
I like to contemplate beer.
But when I start to daydream,
My mind turns straight to wine.

Do I love wine more than beer?

I like to use words about beer.
But when I stop my talking,
My mind turns straight to wine.

I hate bugs and errors.
But I just think back to wine,
And I'm happy once again.

I like to hang out with programming and deep learning.
But when left alone,
My mind turns straight to wine.
'''

In [22]:
# 단락 구분을 없애고 하나의 문자열로 재저장
tokens = text.split() # '\n' 저장
text = ' '.join(tokens)
print(text)

I get on with life as a programmer, I like to contemplate beer. But when I start to daydream, My mind turns straight to wine. Do I love wine more than beer? I like to use words about beer. But when I stop my talking, My mind turns straight to wine. I hate bugs and errors. But I just think back to wine, And I'm happy once again. I like to hang out with programming and deep learning. But when left alone, My mind turns straight to wine.


In [23]:
# 글자 집합
char_vocab = sorted(list(set(text)))
print(char_vocab)

[' ', "'", ',', '.', '?', 'A', 'B', 'D', 'I', 'M', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']


In [24]:
vocab_size = len(char_vocab)
print('글자 집합의 크기: {}'.format(vocab_size))

글자 집합의 크기: 33


In [25]:
char_to_index = dict((c,i) for i, c in enumerate(char_vocab))
print(char_to_index)

{' ': 0, "'": 1, ',': 2, '.': 3, '?': 4, 'A': 5, 'B': 6, 'D': 7, 'I': 8, 'M': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'r': 26, 's': 27, 't': 28, 'u': 29, 'v': 30, 'w': 31, 'y': 32}


In [35]:
length = 11
sequences = []
for i in range(length, len(text)):
    seq = text[i-length:i]  # # 길이 11의 문자열을 지속적으로 만든다.
    sequences.append(seq)
print('총 훈련 샘플의 수: %d' % len(sequences))

총 훈련 샘플의 수: 426


In [36]:
sequences[:10]

['I get on wi',
 ' get on wit',
 'get on with',
 'et on with ',
 't on with l',
 ' on with li',
 'on with lif',
 'n with life',
 ' with life ',
 'with life a']

In [40]:
# 전체 데이터에 대해 정수 인코딩
X = []
for line in sequences:
    temp_X = [char_to_index[char] for char in line]
    X.append(temp_X)

In [41]:
for line in X[:5]:
    print(line)

[8, 0, 16, 14, 28, 0, 24, 23, 0, 31, 18]
[0, 16, 14, 28, 0, 24, 23, 0, 31, 18, 28]
[16, 14, 28, 0, 24, 23, 0, 31, 18, 28, 17]
[14, 28, 0, 24, 23, 0, 31, 18, 28, 17, 0]
[28, 0, 24, 23, 0, 31, 18, 28, 17, 0, 21]


In [42]:
# 예측 대상이 되는 맨 마지막 글자 분리
sequences = np.array(X)
X = sequences[:,:-1]
y = sequences[:,-1]

In [43]:
for line in X[:5]:
    print(line)

[ 8  0 16 14 28  0 24 23  0 31]
[ 0 16 14 28  0 24 23  0 31 18]
[16 14 28  0 24 23  0 31 18 28]
[14 28  0 24 23  0 31 18 28 17]
[28  0 24 23  0 31 18 28 17  0]


In [44]:
print(y[:5])

[18 28 17  0 21]


In [45]:
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]  # X에 대한 원-핫 인코딩
X = np.array(sequences)
y = to_categorical(y, num_classes=vocab_size)

In [46]:
print(X.shape) # (# of samples, input_length, input_dim)

(426, 10, 33)


### 2) 모델 설계하기

In [47]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [48]:
model = Sequential()
model.add(LSTM(80, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))

In [49]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=2)

Epoch 1/100
14/14 - 2s - loss: 3.4534 - accuracy: 0.1432
Epoch 2/100
14/14 - 0s - loss: 3.2640 - accuracy: 0.1972
Epoch 3/100
14/14 - 0s - loss: 3.0315 - accuracy: 0.1972
Epoch 4/100
14/14 - 0s - loss: 2.9747 - accuracy: 0.1972
Epoch 5/100
14/14 - 0s - loss: 2.9524 - accuracy: 0.1972
Epoch 6/100
14/14 - 0s - loss: 2.9331 - accuracy: 0.1972
Epoch 7/100
14/14 - 0s - loss: 2.9111 - accuracy: 0.1972
Epoch 8/100
14/14 - 0s - loss: 2.8964 - accuracy: 0.1972
Epoch 9/100
14/14 - 0s - loss: 2.8703 - accuracy: 0.1972
Epoch 10/100
14/14 - 0s - loss: 2.8416 - accuracy: 0.1972
Epoch 11/100
14/14 - 0s - loss: 2.8120 - accuracy: 0.1995
Epoch 12/100
14/14 - 0s - loss: 2.7941 - accuracy: 0.2230
Epoch 13/100
14/14 - 0s - loss: 2.7401 - accuracy: 0.2160
Epoch 14/100
14/14 - 0s - loss: 2.6851 - accuracy: 0.2371
Epoch 15/100
14/14 - 0s - loss: 2.6238 - accuracy: 0.2512
Epoch 16/100
14/14 - 0s - loss: 2.5757 - accuracy: 0.2324
Epoch 17/100
14/14 - 0s - loss: 2.4999 - accuracy: 0.2700
Epoch 18/100
14/14 - 0s

In [58]:
def sentence_generation(model, char_to_index, seq_length, seed_text, n):
    init_text = seed_text
    sentence = ''
    for _ in range(n):
        encoded = [char_to_index[char] for char in seed_text] # 정수인코딩
        encoded = pad_sequences([encoded], maxlen=seq_length, padding='pre') # 데이터에 대한 테딩
        encoded = to_categorical(encoded, num_classes=len(char_to_index))
        result = model.predict_classes(encoded, verbose=0)
        
        for char, index in char_to_index.items():
            if index == result:
                break
        seed_text = seed_text + char
        sentence = sentence + char
    sentence = init_text + sentence
    return sentence

In [59]:
print(sentence_generation(model, char_to_index, 10, 'I get on w', 80))

I get on with life as a programmer, I like to use words about beer. But when I start to da
